# Case Study : Fitting the BL Lac SED of Mrk 421 #

## Learning Outcome ##

1. *How real data can be plotted along with errorbars*
2. *Use the fitting models available with *agnpy* and apply it on real data*
3. *Finally interpret the results obtained and decipher the properties of jets*

In this case study we shall use the real Mrk 421 data along with errors due to systematics in the observing instruments and fit the same using *Leptonic Model* (i.e., Synchrotron + Synchrotron self-compton model) for Broken Power-Law particle spectra. 
Importantly here we will use observational constraints - *--Variablity time scale--* to estimate the properties of the blob.  

## STEP 1 ##

Load all the required modules from standard python - numpy, astropy and matplotlib. 

From *agnpy* - spectra, plot utilities and *agnpy.fit* the fitting module. 

Finally modules *sherpa* -- (A Hyperparameter optimization toolkit based on ML) - *fit*, *stats* and *optmethods*.

In [7]:
# import numpy, astropy and matplotlib for basic functionalities
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt

# import agnpy classes
from agnpy.spectra import BrokenPowerLaw
from agnpy.fit import SynchrotronSelfComptonModel, load_sherpa_flux_points
from agnpy.utils.plot import load_mpl_rc, sed_y_label

load_mpl_rc()

# import sherpa classes
from sherpa.fit import Fit
from sherpa.stats import Chi2
from sherpa.optmethods import LevMar

ImportError: cannot import name 'SynchrotronSelfComptonModel' from 'agnpy.fit' (C:\Users\thesi\AppData\Local\Programs\Python\Python311\Lib\site-packages\agnpy\fit\__init__.py)

## STEP 2 ##

Define the properties of the electron spectra as Broken Power Law
$$N(\gamma') [cm^{-3}] = k \left[
\left(\frac{\gamma'}{\gamma'_b}\right)^{-p_1} \, H(\gamma'; \gamma'_{\rm min}, \gamma'_b) +
\left(\frac{\gamma'}{\gamma'_b}\right)^{-p_2} \, H(\gamma'; \gamma'_{b}, \gamma'_{\rm max})
\right]$$

where $H(\gamma)$ is a Heavy-side function used to ensure continuity of the spectra.

Finally, define the leptonic model based on Synchrotron (for the first hump) and SSC (for the second hump). Here the important part is to also define the *backend* as sherpa for doing the model fits. 


In [8]:
# electron energy distribution
n_e = BrokenPowerLaw(
    k=1e-8 * u.Unit("cm-3"),
    p1=2.02,
    p2=3.43,
    gamma_b=1e5,
    gamma_min=500,
    gamma_max=1e6,
)

# initialise the sherpa model
ssc_model = SynchrotronSelfComptonModel(n_e, backend="sherpa")

NameError: name 'SynchrotronSelfComptonModel' is not defined

## STEP 3 ##

Now give initial constraints to the model based on observations like redshift $z$, approximate Doppler Factor $\delta_D$ and variablity time scale $t_{\rm var}$ and some initial guess/value of magnetic field $B$.

The size of blob can be obtained using variablity time scale and other quantities as $$ R_b = \frac{c t_{\rm var} \delta_D}{1 + z}$$

The advantage of using *sherpa* backend is visible when the model is printed. You can see the range of values that would be used for modelling the different parameter space. 

In [9]:
ssc_model.z = 0.0308
ssc_model.delta_D = 18
ssc_model.t_var = (1 * u.d).to_value("s")
ssc_model.t_var.freeze()
ssc_model.log10_B = -1.3

NameError: name 'ssc_model' is not defined

In [10]:
ssc_model

NameError: name 'ssc_model' is not defined

## STEP 4a ## 

Get the data from the *ecsv* file (a simple CSV file with meta-data/header). Importantly the flux data from each instrument has error associated with it and this needs to be accounted during fitting process. The dictionary containing systematics is given. 

Choose a energy range from $E_{\rm min}$ = h 10$^{11}$ Hz [in eV] to $E_{\rm max}$ = 100 TeV and obtain the data. 

In [ ]:
systematics_dict = {
    "Fermi": 0.10,
    "GASP": 0.05,
    "GRT": 0.05,
    "MAGIC": 0.30,
    "MITSuME": 0.05,
    "Medicina": 0.05,
    "Metsahovi": 0.05,
    "NewMexicoSkies": 0.05,
    "Noto": 0.05,
    "OAGH": 0.05,
    "OVRO": 0.05,
    "RATAN": 0.05,
    "ROVOR": 0.05,
    "RXTE/PCA": 0.10,
    "SMA": 0.05,
    "Swift/BAT": 0.10,
    "Swift/UVOT": 0.05,
    "Swift/XRT": 0.10,
    "VLBA(BK150)": 0.05,
    "VLBA(BP143)": 0.05,
    "VLBA(MOJAVE)": 0.05,
    "VLBA_core(BP143)": 0.05,
    "VLBA_core(MOJAVE)": 0.05,
    "WIRO": 0.05,
}

# define minimum and maximum energy to be used in the fit
E_min = (1e11 * u.Hz).to("eV", equivalencies=u.spectral())
E_max = 100 * u.TeV

sed_path = 'data/Mrk421_2011.ecsv'
sed = load_sherpa_flux_points(sed_path, E_min, E_max, systematics_dict)


## Step 4b ##

Now plot the SED data along with the initial model based on the initial guess of the parameters.


In [ ]:
# array of energies to plot the model
E = np.logspace(np.log10(E_min.to_value("eV")), np.log10(E_max.to_value("eV")), 100)

fig, ax = plt.subplots(figsize=(8, 6))

ax.errorbar(
    sed.x,
    sed.y,
    yerr=sed.get_error(),
    marker=".",
    ls="",
    color="k",
    label="Abdo et al. (2011)",
)
ax.loglog(E, ssc_model(E), ls="-", color="crimson", label="SSC model")
ax.set_ylabel(sed_y_label)
ax.set_xlabel(r"$E\,/\,{\rm eV}$")
ax.legend()

## Step 5 ##

Finally define a fitter function with inputs as follows 

1. Observation data
2. Initial Model
3. Statistics to check - Here is Chi-Square statistics for goodness of Fit.
4. Method of Optimization - Here is Levenberg-Marquardt optimization (often referred to as LevMar) is a numerical optimization technique used to solve nonlinear least-squares problems.

In [ ]:
%%time
fitter = Fit(sed, ssc_model, stat=Chi2(), method=LevMar())

# perform the fit and time it!
results = fitter.fit()
print("Fit succesful = ", results.succeeded)
print(results.format())

## Step 6 ##

Finally overplot the best fit model with the data and observe the difference. 
Also print the final Model values and interpret the jet parameters.


In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

ax.errorbar(
    sed.x,
    sed.y,
    yerr=sed.get_error(),
    marker=".",
    ls="",
    color="k",
    label="Abdo et al. (2011)",
)
ax.loglog(E, ssc_model(E), ls="-", color="crimson", label="SSC model")
ax.set_ylabel(sed_y_label)
ax.set_xlabel(r"$E\,/\,{\rm eV}$")
ax.legend()

plt.show()

In [ ]:
ssc_model

## TASK ##

1. Mrk 421 is a typical Blazar and therefore has an incident angle $\approx 0$ implying bulk Lorentz factor $\Gamma \approx \delta_D$. 
From the best model can you estimate the jet kinetic and jet magnetic power? 

2. Can you attempt to fit the same data with a power-law distribution instead of a Broken power-law distribution? What values of Doppler factor and Jet power you obtain in that case.

3. Can the simple Leptonic model based on Synchroton Self-Compton used above either with Broken Power-Law or Simple Power-Law be applied to a FSRQ source PKSPKS1510.? See the *AGNPY Documentation* - https://agnpy.readthedocs.io/en/latest/tutorials/ec_dt_sherpa_fit.html